In [1]:
import pickle as pkl
import pandas as pd
with open("SelectedJobData_0801-1001.pkl", "rb") as f:
    object = pkl.load(f)
    
df = pd.DataFrame(object)
df.to_csv(r'SelectedJobData_0801-1001.csv')

In [ ]:
import csv
import numpy as np
import pandas as pd
import datetime as dt

# extract fields
fields = ['_id', 'group', 'owner', 'job_name', 'job_number', 'slots', 'mem', 'submission_time', 'start_time', 'end_time']
df = pd.read_csv('data/0_raw_data.csv', skipinitialspace=True, usecols=fields)

# save to
df.to_csv(r'data/1_selection_data.csv', index = False)

df


#### Show difference in time

In [3]:
# remove the incorrect records ------------------

df = pd.read_csv("data/1_selection_data.csv", parse_dates=True)

# Tq = log( start_time - submit_time )
df["queuetime"] = (pd.to_datetime(df["start_time"], format='%Y-%m-%d %H:%M:%S') - pd.to_datetime(df["submission_time"], format='%Y-%m-%d %H:%M:%S')).dt.total_seconds()

# Tr = log( end_time - start_time )
df["runtime"] = (pd.to_datetime(df["end_time"], format='%Y-%m-%d %H:%M:%S') - pd.to_datetime(df["start_time"], format='%Y-%m-%d %H:%M:%S')).dt.total_seconds()

# Remove negative records and convert 0 -> 0.[0]1 ------------------

df = df[(df.mem >= 0.0) & (df.slots >= 0.0) & (df.queuetime >= 0.0) & (df.runtime >= 0.0)]

df.to_csv(r'data/2_non_neg.csv', index = False)

print(df.columns.tolist())

['_id', 'group', 'owner', 'job_name', 'job_number', 'submission_time', 'start_time', 'end_time', 'mem', 'slots', 'queuetime', 'runtime']


In [4]:
# example of scaling

from pandas import read_csv
from pandas import DataFrame
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler

attr = ['mem', 'slots', 'queuetime', 'runtime']

df["slots"] = df["slots"] / 36

# retrieve just the numeric input values
data = df[attr].to_numpy()

# perform a robust scaler transform of the dataset
trans = StandardScaler()
data = trans.fit_transform(data)

# convert the array back to a dataframe
dataset = DataFrame(data)

features = ['N_n', 'M_u', 'T_q', 'T_r']

dataset.columns = features

dataset.to_csv(r'data/scaled.csv', index = False)

#### Just data

In [ ]:
from pandas import read_csv
from pandas import DataFrame
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("data/2_non_neg.csv", parse_dates=True)

attr = ['mem', 'slots', 'queuetime', 'runtime']

df["slots"] = df["slots"] / 36

# retrieve just the numeric input values
data = df[attr].to_numpy()

# convert the array back to a dataframe
dataset = DataFrame(data)

features = ['N_n', 'M_u', 'T_q', 'T_r']

dataset.columns = features

dataset.to_csv(r'data/data.csv', index = False)

#### Log

In [6]:
import numpy as np

# read data
df = pd.read_csv("data/2_non_neg.csv", parse_dates=True)

df = df.replace([0],0.0000000001)

# Nn = log( nproc/Cn )
df["N_n"] = np.log(df["slots"]/36)

# Mu = log( mem_used )
df["M_u"] = np.log(df["mem"])

# Tq = log( start_time - submit_time )
df["T_q"] = np.log(df["queuetime"])

# Tr = log( end_time - start_time )
df["T_r"] = np.log(df["runtime"])

df.drop(['submission_time', 'start_time', 'end_time', 'mem', 'slots', 'queuetime', 'runtime'], inplace=True, axis=1)

df.to_csv(r'data/log_data.csv', index = False)

print(df.columns.tolist())


['_id', 'group', 'owner', 'job_name', 'job_number', 'N_n', 'M_u', 'T_q', 'T_r']


#### Helper functions

In [ ]:
# find the lowest non-zero
df[df['mem'].gt(0)].min()

In [ ]:
# subtract date time
import datetime as dt

start = job_accounting["start_time"][0]
submit = job_accounting["submission_time"][0]
print(start, submit)

a = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
b = dt.datetime.strptime(submit, '%Y-%m-%d %H:%M:%S')

print(a-b)

In [ ]:
import pandas as pd
# Smaller data

rows = 5000

df1 = pd.read_csv("data/2_non_neg.csv", skipinitialspace=True)
df1['id'] = df1['group'] + "_" + df1['_id'].str.slice(-5,-1)  
df1a = df1['id'].head(rows)

df2 = pd.read_csv("data/data_all.csv", skipinitialspace=True)
df2 = df2.head(rows)

df3 = pd.concat([df1a, df2], axis=1, ignore_index=True)
df3.columns = ['name', 'N_n', 'M_u', 'T_q', 'T_r']

df3.to_csv(r'data/data.csv', index = False)



In [ ]:
import pandas as pd

df = pd.read_csv('data/1_selection_data.csv', skipinitialspace=True)

df['_id'][1][-5:-1]
# pd.unique(df['owner']).size

In [ ]:
df = pd.read_csv('data/data.csv', skipinitialspace=True)

df[~df.name.str.contains('\w')].name.count()